# Build a Chatbot with Keras
This notebook walks you through building a simple chatbot enabled model using Keras. We will also be using [NLTK](https://www.nltk.org/) which is a natural language toolkit

In [3]:
import nltk
nltk.download('punkt')

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jdeha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


First we import our intents data. Our intents are made of 3 parts: **the tag** which is the intent we want to match, **patterns** natural language ways to learn which intent to match to, **context** to allow connecting with other intents(tags)

In [5]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

We now need to parse out data. The tags are our labels and the patterns are our training data to learn from in order to get to the right tag

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes")
print(classes)
print()
print (len(words), "unique stemmed words")
print(words)

32 documents
7 classes
['book_search', 'goodbye', 'greeting', 'options', 'renew_book', 'return_book', 'thanks']
55 unique stemmed words
["'s", ',', 'a', 'adv', 'al', 'am', 'anyon', 'ar', 'awesom', 'be', 'book', 'bye', 'can', 'chat', 'could', 'day', 'do', 'for', 'giv', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'lat', 'lik', 'list', 'look', 'me', 'my', 'new', 'next', 'nic', 'of', 'off', 'provid', 'renew', 'return', 'see', 'show', 'som', 'support', 'thank', 'that', 'ther', 'tim', 'to', 'until', 'want', 'what', 'would', 'you']


Here we will loop through the patterns and use the stemmer to create a base word in an attempt to represent related words and then create a set of features (patterns) and labels (tags)


In [7]:
training = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

Let's create our Keras model architecture. Note that the last layer matches the number of classes we learn on.

In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

W1122 22:12:17.314031 16224 deprecation_wrapper.py:119] From C:\Users\jdeha\Anaconda3\envs\keras\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1122 22:12:17.329069 16224 deprecation_wrapper.py:119] From C:\Users\jdeha\Anaconda3\envs\keras\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1122 22:12:17.331033 16224 deprecation_wrapper.py:119] From C:\Users\jdeha\Anaconda3\envs\keras\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1122 22:12:17.347061 16224 deprecation_wrapper.py:119] From C:\Users\jdeha\Anaconda3\envs\keras\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1

In [9]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

W1122 22:12:20.151536 16224 deprecation_wrapper.py:119] From C:\Users\jdeha\Anaconda3\envs\keras\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1122 22:12:20.160536 16224 deprecation_wrapper.py:119] From C:\Users\jdeha\Anaconda3\envs\keras\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [10]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

W1122 22:12:21.350064 16224 deprecation.py:323] From C:\Users\jdeha\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
32/32 [==============================] - 0s 13ms/step - loss: 1.9254 - acc: 0.1563
Epoch 2/200
32/32 [==============================] - 0s 375us/step - loss: 1.9329 - acc: 0.1563
Epoch 3/200
32/32 [==============================] - 0s 437us/step - loss: 1.8023 - acc: 0.2500
Epoch 4/200
32/32 [==============================] - 0s 344us/step - loss: 1.8282 - acc: 0.2188
Epoch 5/200
32/32 [==============================] - 0s 406us/step - loss: 1.6137 - acc: 0.4688
Epoch 6/200
32/32 [==============================] - 0s 345us/step - loss: 1.4993 - acc: 0.3750
Epoch 7/200
32/32 [==============================] - 0s 343us/step - loss: 1.4317 - acc: 0.4688
Epoch 8/200
32/32 [==============================] - 0s 375us/step - loss: 1.2604 - acc: 0.6250
Epoch 9/200
32/32 [==============================] - 0s 375us/step - loss: 1.1976 - acc: 0.6250
Epoch 10/200
32/32 [==============================] - 0s 1ms/step - loss: 1.0898 - acc: 0.7188
Epoch 11/200
32/32 [======================

32/32 [==============================] - 0s 344us/step - loss: 0.0044 - acc: 1.0000
Epoch 169/200
32/32 [==============================] - 0s 313us/step - loss: 0.0079 - acc: 1.0000
Epoch 170/200
32/32 [==============================] - 0s 343us/step - loss: 0.0078 - acc: 1.0000
Epoch 171/200
32/32 [==============================] - 0s 344us/step - loss: 0.0497 - acc: 0.9688
Epoch 172/200
32/32 [==============================] - 0s 375us/step - loss: 0.0041 - acc: 1.0000
Epoch 173/200
32/32 [==============================] - 0s 313us/step - loss: 0.0122 - acc: 1.0000
Epoch 174/200
32/32 [==============================] - 0s 405us/step - loss: 0.0038 - acc: 1.0000
Epoch 175/200
32/32 [==============================] - 0s 281us/step - loss: 0.0073 - acc: 1.0000
Epoch 176/200
32/32 [==============================] - 0s 281us/step - loss: 0.0131 - acc: 1.0000
Epoch 177/200
32/32 [==============================] - 0s 281us/step - loss: 0.0286 - acc: 1.0000
Epoch 178/200
32/32 [=============

In [15]:
model.save("chatbot_model.h5")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, 
            open( "chatbot-data.pkl", "wb" ) )